In [1]:
# !pip install -q accelerate==0.21.0 transformers==4.31.0 xformers bitsandbytes==0.40.2  # peft==0.4.0
# !pip install einops  # for falcon model
# !pip install torch

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, logging, BitsAndBytesConfig
import torch, re

logging.set_verbosity(logging.CRITICAL)

In [3]:
llama2 = 'NousResearch/Llama-2-7b-chat-hf'
falcon7b = 'tiiuae/falcon-7b-instruct'

# Load the entire model on the GPU 0
DEVICE = {"": 0}  # or try 'auto'
print(DEVICE)

{'': 0}


In [4]:
def load_model(model_name):
    # NOTE: BitsAndBytesConfig might be unnecessary. I used it as an attempt to get falcon to load in colab.
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        low_cpu_mem_usage=True,  # Auto set to true when specifying device_map
        return_dict=True,
        torch_dtype=torch.float16,
        device_map=DEVICE,  # device_map
        trust_remote_code=True,
        quantization_config=bnb_config
        # load_in_4bit=True,  # unsure if this should maybe help with loading it
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    return model, tokenizer

    
def predict_model(model, tokenizer, med_type, context):
    # Run text generation pipeline with our next model
    prompt = f"Write me a {med_type} meditation script of about 200 words which {GOALS[med_type]}"

    if model_name == llama2:
        tokenized_prompt = f"""<s>[INST] <<SYS>>\n{context}\n<</SYS>>\n\n{prompt} [/INST]"""
    elif model_name == falcon7b:
        # Do not have a stray space at end of it will mess with tokenization
        tokenized_prompt = f"""### Instruction:\n{prompt}\n\n### Context:\n{context}\n\n### Response:"""
    else:
        raise Exception('Invalid model name')

    encodeds = tokenizer(tokenized_prompt, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to("cuda:0")

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768,
        top_p=0.95,  # Added. Seems to work!
        top_k=1000,  # Added
        do_sample=True)
    # print('encoded\n', generated_ids)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    # print('Prompt: ', tokenized_prompt, '\n')
    output = decoded[0]
    
    # Extract the actual script (remove the prompt)
    if model_name == llama2:
        script = re.sub(r'[\s\S]*\[/INST\].*', '', output).lstrip()
        # ^ Any char followed by [/INST] followed by any non-newline char (.*)
        # The rest of the line usually says 'Great! Here's a meditation script that ...
        print(f'### PROMPT:\n{tokenized_prompt}\n\n')
    else:  # model_name == falcon7b:
        script = re.sub(r'.*### Response:', '', output)

    # If response did not end in full sentence, remove the last substring that does not end in a period
    last_period_idx = script.rfind('.')
    if last_period_idx != -1:
        script = script[:last_period_idx + 1]  # + 1 to keep the period

    # print(output)
    # print('\n\nSEPARATION')
    print('Context:\n', context, '\n')
    print('Transcript\n', script)

#### Setting parameters

Change MED_TYPE to any key in the `goals` dictionary. If you'd like, you can add 'goals' as the values to incorporate into the prompt. You can also leave them empty and it should run fine.  
Also you can set CONTEXT to a string that a user would provide to our LLM. There are already some commented examples.

In [5]:
GOALS = {
    'mindful observation': 'emphasizes the practice of mindfully observing oneself in a state of stillness, which includes lying meditation, sitting in silence, and observing thoughts and emotions.',
    'body-centered meditation': 'focuses on heightened awareness of the body, breath, and sensory perceptions, sometimes involving concentration on specific body locations or energy centers.',
    'visual concentration': 'involves techniques like visualizations and concentration on visual objects to achieve a centered state of mind.',
    'contemplation': 'encourages deep reflection on contradictions, paradoxes, or spiritual questions.',
    'affect-centered meditation': 'encompasses practices that cultivate positive emotions such as compassion, loving-kindness, and equanimity.',
    'mantra meditation': 'involves the repetitive use of syllables, words, or phrases, often with sounds and mantras, to facilitate meditation.',
    'movement meditation': 'combines mindfulness with physical activity, including manipulating the breath, walking, and sensory observation.'
}

#### Model falcon7b

In [6]:
model_name = falcon7b
model, tokenizer = load_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
med_type = 'mindful observation'

context = "I'm sad about losing a lost one. I need comfort"  # Nice: I noticed llama did address the loss
context += "\nMake sure the response ends with a complete sentence."

predict_model(model, tokenizer, med_type, context)

Context:
 I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence. 

Transcript
 ### Instruction:
Write me a mindful observation meditation script of about 200 words which emphasizes the practice of mindfully observing oneself in a state of stillness, which includes lying meditation, sitting in silence, and observing thoughts and emotions.

### Context:
I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence.


I'm sad about losing a loved one, and I need comfort. To bring me relief, I try to make myself mindful of the here and now and of my breath. As I focus on my breath, I watch my thoughts float by without attaching to them. There's a strange stillness in my mind, where I can almost feel emptiness and infinite space. If I become caught up in my thoughts, I let them pass without judgment and try to gently turn back to my breath. I find solace in the sense of presence and the acceptance of empti

In [8]:
med_type = 'body-centered meditation'

context = "I'm sad about losing a lost one. I need comfort"  # Nice: I noticed llama did address the loss
context += "\nMake sure the response ends with a complete sentence."

predict_model(model, tokenizer, med_type, context)

Context:
 I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence. 

Transcript
 ### Instruction:
Write me a body-centered meditation meditation script of about 200 words which focuses on heightened awareness of the body, breath, and sensory perceptions, sometimes involving concentration on specific body locations or energy centers.

### Context:
I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence.


As I sit here thinking of you, I remember all the good times and the little ways you made my days a bit brighter. I allow the sadness to wash over me, feeling a connection to the warmth and comfort that I had while spent with you. I take a deep breath and reach for my breath, extending the love and comfort from you that your body brought into your life. As I inhale, I acknowledge the sadness in a way that honors you and I, the way your life did. The practice of breathing deeply and taking care of 

#### Model llama2

In [6]:
model_name = llama2
model, tokenizer = load_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
med_type = 'mindful observation'

context = "I'm sad about losing a lost one. I need comfort"  # Nice: I noticed llama did address the loss
context += "\nMake sure the response ends with a complete sentence."

predict_model(model, tokenizer, med_type, context)

/home/javiferfer/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


### PROMPT:
<s>[INST] <<SYS>>
I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence.
<</SYS>>

Write me a mindful observation meditation script of about 200 words which emphasizes the practice of mindfully observing oneself in a state of stillness, which includes lying meditation, sitting in silence, and observing thoughts and emotions. [/INST]


Context:
 I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence. 

Transcript
 Find a quiet and comfortable place to sit, either lying down or sitting in silence. Take a few deep breaths and focus your attention on your body, starting from your toes and moving up to the top of your head. As you do this, notice any areas of tension or discomfort and try to let go of them.

As you settle into stillness, bring your attention to your breath. Notice the sensation of the air moving in and out of your body, and try to let go of any thoughts or emotions that 

In [8]:
med_type = 'body-centered meditation'

context = "I'm sad about losing a lost one. I need comfort"  # Nice: I noticed llama did address the loss
context += "\nMake sure the response ends with a complete sentence."

predict_model(model, tokenizer, med_type, context)

### PROMPT:
<s>[INST] <<SYS>>
I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence.
<</SYS>>

Write me a body-centered meditation meditation script of about 200 words which focuses on heightened awareness of the body, breath, and sensory perceptions, sometimes involving concentration on specific body locations or energy centers. [/INST]


Context:
 I'm sad about losing a lost one. I need comfort
Make sure the response ends with a complete sentence. 

Transcript
 Find a comfortable seated position, either on a cushion on the floor or in a chair. Close your eyes and bring your attention to your breath, feeling the sensation of the breath moving in and out of your body. Allow your attention to settle on the sensation of the breath, and let go of any thoughts or distractions that arise.

As you continue to focus on your breath, begin to bring your attention to different parts of your body. Starting at your toes, bring your awareness up throug